# textRNN

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

In [18]:
sentence = "if you want me"

# make dictionary
char_set = list(set(sentence))
char_dic = {c:i for i, c in enumerate(char_set)}
print(char_dic)

{'a': 0, 'w': 1, 'm': 2, 'f': 3, 't': 4, 'y': 5, 'n': 6, 'o': 7, 'i': 8, 'e': 9, 'u': 10, ' ': 11}


In [3]:
## predict next word
class TextRNN(nn.Module):
    def __init__(self, input_size, hidden_size, n_layers):
        super(TextRNN, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, n_layers, batch_first = True)
        
    def forward(self, X, hidden):
        outputs, _status = self.rnn(X, hidden)
        return outputs, _status

In [5]:
input_size = len(char_dic) # input 시 들어가는 character 수(vocab size)
hidden_size = len(char_dic) # output 시 나오는 character 수(number of class. 지금은 다음 단어 예측하는 거니까 vocab size 와 동일))

In [6]:
sentence_idx = [char_dic[c] for c in sentence]
x_data = [sentence_idx[:-1]] # 마지막 글자 빼고 전부 input
x_data = [np.eye(len(char_dic))[x] for x in x_data] # one hot encoding
y_data = [sentence_idx[1:]] # 첫번째부터 마지막 까지 output

In [7]:
# transform as torch tensor
X = torch.FloatTensor(x_data)
y = torch.LongTensor(y_data)

In [8]:
# declare RNN
rnn = TextRNN(input_size, hidden_size, 1)

# loss& optimizer setting
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(rnn.parameters(), lr = 0.001)

In [20]:
epochs = 1000

for epoch in range(epochs):
    optimizer.zero_grad()
    
    hidden = torch.zeros(1, 1, input_size)
    outputs, _status = rnn(X, hidden)
    loss = criterion(outputs.view(-1, input_size), y.view(-1))
    loss.backward()
    optimizer.step()
    
    pred = outputs.data.numpy().argmax(axis = 2)
    pred_str = ''.join([char_set[c] for c in np.squeeze(pred)])
    if (epoch+1) % 100 == 0:
        print('Epoch {}/{}, loss : {:.4f}, pred str : {}'.format(epoch+1, epochs, loss, pred_str))

Epoch 100/1000, loss : 0.9692, pred str : f you want me
Epoch 200/1000, loss : 0.9658, pred str : f you want me
Epoch 300/1000, loss : 0.9629, pred str : f you want me
Epoch 400/1000, loss : 0.9602, pred str : f you want me
Epoch 500/1000, loss : 0.9579, pred str : f you want me
Epoch 600/1000, loss : 0.9558, pred str : f you want me
Epoch 700/1000, loss : 0.9539, pred str : f you want me
Epoch 800/1000, loss : 0.9521, pred str : f you want me
Epoch 900/1000, loss : 0.9506, pred str : f you want me
Epoch 1000/1000, loss : 0.9491, pred str : f you want me


In [17]:
outputs.shape

torch.Size([1, 13, 12])

In [15]:
outputs[:,-1] # hidden_state를 거쳐 나오는 모든 output. shape = [batch_size, time_stemps, n_class]]

tensor([[-0.9316, -0.8163, -0.9825, -0.9831, -0.8880, -0.9875, -0.8529, -0.8410,
         -0.9766,  0.8771, -0.9913, -0.9711]], grad_fn=<SelectBackward>)

In [16]:
_status # 가장 마지막 hidden state를 거쳐 나온 값. output[:,-1]과 같음.shape = [batch_size, 1, n_class]

tensor([[[-0.9316, -0.8163, -0.9825, -0.9831, -0.8880, -0.9875, -0.8529,
          -0.8410, -0.9766,  0.8771, -0.9913, -0.9711]]],
       grad_fn=<StackBackward>)